In [1]:
import pyro
import torch
from pyro import poutine
import pandas as pd
from pyro.distributions import MultivariateNormal as MVN
from pyro.distributions import HalfCauchy
from pyro.contrib.autoguide import AutoDelta
from pyro.infer import SVI, Trace_ELBO

In [2]:
data = pd.read_csv("dataset/toyset/processed_toyset.csv")
training = data.iloc[0:1000,]
time_gap = torch.tensor(training["time_gap"].values)
training.head()

,timestamp,gbpjpy,gbpusd,usdjpy,time_gap
0,2019-05-01 00:00:00,4.97986,0.265659,4.713935,1
1,2019-05-01 00:01:00,4.97986,0.265659,4.713935,1
2,2019-05-01 00:02:00,4.97986,0.265659,4.713935,1
3,2019-05-01 00:03:00,4.97986,0.265659,4.713935,1
4,2019-05-01 00:04:00,4.97986,0.265659,4.713935,1


We will denote observed exchange rates as y and latent currency intrinsic values as x

In [3]:
colnames = list(training)[1:-1]
num_exchange = len(colnames)
y_mu = torch.zeros(num_exchange)

first_curr = [exchange[0:3] for exchange in colnames]
second_curr = [exchange[3:6] for exchange in colnames]
all_curr = list(set(first_curr + second_curr))
num_currency = len(all_curr)
x_mu = torch.zeros(num_currency)

meas_matrix = []
for i in range(num_exchange):
    temp_tens = torch.zeros(num_currency)
    temp_tens[all_curr.index(first_curr[i])] = 1
    temp_tens[all_curr.index(second_curr[i])] = -1
    meas_matrix.append(temp_tens)
meas_matrix = torch.cat(meas_matrix).view(num_exchange, -1)
meas_matrix

tensor([[ 1.,  0., -1.],
        [ 1., -1.,  0.],
        [ 0.,  1., -1.]])

In [4]:
data = torch.tensor(training.iloc[:,1:-1].values)
data

tensor([[4.9799, 0.2657, 4.7139],
        [4.9799, 0.2657, 4.7139],
        [4.9799, 0.2657, 4.7139],
        ...,
        [4.9802, 0.2690, 4.7109],
        [4.9801, 0.2690, 4.7109],
        [4.9802, 0.2689, 4.7110]], dtype=torch.float64)

In [26]:
def model(data):
    num_proc_L_theta = int(num_currency ** 2)
    proc_L_theta = pyro.sample("proc_prob",
                               pyro.distributions.HalfCauchy(0.1 * torch.ones(num_proc_L_theta))
                              )
    proc_L = torch.tril(proc_L_theta.view(num_currency, -1))

    num_meas_L_theta = int(num_exchange ** 2)
    meas_L_theta = pyro.sample("meas_prob",
                              pyro.distributions.HalfCauchy(0.1 * torch.ones(num_meas_L_theta)))
    meas_L = torch.tril(meas_L_theta.view(num_exchange, -1))
    
    x = torch.zeros(num_currency)
    seq_length = data.shape[0]
    for t in pyro.markov(range(seq_length)):
        x = pyro.sample("x_{}".format(t), MVN(x, scale_tril = time_gap[t]**0.5 * proc_L))
        print("Latent state")
        print(x)
        y = pyro.sample("y_{}".format(t), MVN(torch.mv(meas_matrix, x), scale_tril = meas_L), obs = data[t])
        print("Observed")
        print(data[t])
        print(y)

In [27]:
guide = AutoDelta(model)
optimizer = pyro.optim.Adam({"lr": 0.0005})
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

In [28]:
svi.step(data)

Latent state
tensor([-0.0252, -0.0372, -0.0545], requires_grad=True)
Observed
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
Latent state
tensor([-0.0573, -0.1321, -0.1282], requires_grad=True)
Observed
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
Latent state
tensor([-0.0016, -0.0909, -0.1431], requires_grad=True)
Observed
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
Latent state
tensor([-0.0174, -0.1645, -0.1987], requires_grad=True)
Observed
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
Latent state
tensor([ 0.0100, -0.0716, -0.2236], requires_grad=True)
Observed
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
Latent state
tensor([ 0.0543,  0.0013, -0.1612], requires_grad=True)
O

Latent state
tensor([-0.7249, -1.8744, -0.8793], requires_grad=True)
Observed
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
Latent state
tensor([-0.7983, -2.0759, -0.8916], requires_grad=True)
Observed
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
Latent state
tensor([-0.7882, -2.0413, -0.8819], requires_grad=True)
Observed
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
Latent state
tensor([-0.7463, -1.9928, -0.8575], requires_grad=True)
Observed
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
Latent state
tensor([-0.7930, -2.0274, -0.8984], requires_grad=True)
Observed
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
Latent state
tensor([-0.7399, -1.8427, -0.8328], requires_grad=True)
O

Latent state
tensor([-0.2236, -1.0035, -0.8015], requires_grad=True)
Observed
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
Latent state
tensor([-0.1592, -0.9249, -0.7853], requires_grad=True)
Observed
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
Latent state
tensor([-0.1816, -0.9869, -0.8100], requires_grad=True)
Observed
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
Latent state
tensor([-0.1776, -0.9156, -0.7874], requires_grad=True)
Observed
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
Latent state
tensor([-0.1330, -0.8154, -0.7042], requires_grad=True)
Observed
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
Latent state
tensor([-0.1716, -0.9614, -0.7393], requires_grad=True)
O

Latent state
tensor([-0.0856, -0.9282, -0.8589], requires_grad=True)
Observed
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
Latent state
tensor([ 0.0467, -0.8034, -0.8749], requires_grad=True)
Observed
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
Latent state
tensor([ 0.0312, -0.7562, -0.8230], requires_grad=True)
Observed
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
Latent state
tensor([-0.0470, -0.8900, -0.8234], requires_grad=True)
Observed
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
Latent state
tensor([-0.0346, -0.8690, -0.8637], requires_grad=True)
Observed
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
Latent state
tensor([-0.1253, -0.9509, -0.8349], requires_grad=True)
O

Latent state
tensor([-0.2847, -0.8489, -0.8808], requires_grad=True)
Observed
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
Latent state
tensor([-0.2210, -0.6873, -0.8613], requires_grad=True)
Observed
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
Latent state
tensor([-0.1635, -0.6381, -0.8527], requires_grad=True)
Observed
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
Latent state
tensor([-0.2661, -0.7990, -0.8957], requires_grad=True)
Observed
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
Latent state
tensor([-0.2981, -0.9275, -0.9652], requires_grad=True)
Observed
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
tensor([4.9799, 0.2657, 4.7139], dtype=torch.float64)
Latent state
tensor([-0.2384, -0.7918, -0.8973], requires_grad=True)
O

...

Observed
tensor([4.9805, 0.2669, 4.7133], dtype=torch.float64)
tensor([4.9805, 0.2669, 4.7133], dtype=torch.float64)
Latent state
tensor([1.1563, 2.9109, 1.0591], grad_fn=<ExpandBackward>)
Observed
tensor([4.9805, 0.2670, 4.7132], dtype=torch.float64)
tensor([4.9805, 0.2670, 4.7132], dtype=torch.float64)
Latent state
tensor([1.2508, 3.1154, 1.0613], grad_fn=<ExpandBackward>)
Observed
tensor([4.9805, 0.2671, 4.7132], dtype=torch.float64)
tensor([4.9805, 0.2671, 4.7132], dtype=torch.float64)
Latent state
tensor([1.2409, 3.1327, 1.0102], grad_fn=<ExpandBackward>)
Observed
tensor([4.9809, 0.2674, 4.7132], dtype=torch.float64)
tensor([4.9809, 0.2674, 4.7132], dtype=torch.float64)
Latent state
tensor([1.2036, 3.1070, 1.0459], grad_fn=<ExpandBackward>)
Observed
tensor([4.9808, 0.2672, 4.7133], dtype=torch.float64)
tensor([4.9808, 0.2672, 4.7133], dtype=torch.float64)
Latent state
tensor([1.1787, 3.1067, 1.0674], grad_fn=<ExpandBackward>)
Observed
tensor([4.9808, 0.2673, 4.7132], dtype=torch.f

tensor([4.9806, 0.2677, 4.7127], dtype=torch.float64)
Latent state
tensor([1.6600, 3.8916, 1.1878], grad_fn=<ExpandBackward>)
Observed
tensor([4.9806, 0.2676, 4.7127], dtype=torch.float64)
tensor([4.9806, 0.2676, 4.7127], dtype=torch.float64)
Latent state
tensor([1.6551, 3.8892, 1.1832], grad_fn=<ExpandBackward>)
Observed
tensor([4.9805, 0.2676, 4.7127], dtype=torch.float64)
tensor([4.9805, 0.2676, 4.7127], dtype=torch.float64)
Latent state
tensor([1.6682, 3.9379, 1.1736], grad_fn=<ExpandBackward>)
Observed
tensor([4.9804, 0.2675, 4.7126], dtype=torch.float64)
tensor([4.9804, 0.2675, 4.7126], dtype=torch.float64)
Latent state
tensor([1.6806, 3.9134, 1.1483], grad_fn=<ExpandBackward>)
Observed
tensor([4.9805, 0.2676, 4.7126], dtype=torch.float64)
tensor([4.9805, 0.2676, 4.7126], dtype=torch.float64)
Latent state
tensor([1.6258, 3.7012, 1.0122], grad_fn=<ExpandBackward>)
Observed
tensor([4.9805, 0.2675, 4.7127], dtype=torch.float64)
tensor([4.9805, 0.2675, 4.7127], dtype=torch.float64)
L

Latent state
tensor([1.7990, 3.4153, 0.7926], grad_fn=<ExpandBackward>)
Observed
tensor([4.9802, 0.2685, 4.7114], dtype=torch.float64)
tensor([4.9802, 0.2685, 4.7114], dtype=torch.float64)
Latent state
tensor([1.7967, 3.3700, 0.7391], grad_fn=<ExpandBackward>)
Observed
tensor([4.9802, 0.2685, 4.7114], dtype=torch.float64)
tensor([4.9802, 0.2685, 4.7114], dtype=torch.float64)
Latent state
tensor([1.7791, 3.3553, 0.7149], grad_fn=<ExpandBackward>)
Observed
tensor([4.9801, 0.2686, 4.7113], dtype=torch.float64)
tensor([4.9801, 0.2686, 4.7113], dtype=torch.float64)
Latent state
tensor([1.8183, 3.4204, 0.7124], grad_fn=<ExpandBackward>)
Observed
tensor([4.9802, 0.2685, 4.7114], dtype=torch.float64)
tensor([4.9802, 0.2685, 4.7114], dtype=torch.float64)
Latent state
tensor([1.8012, 3.3768, 0.6819], grad_fn=<ExpandBackward>)
Observed
tensor([4.9800, 0.2684, 4.7114], dtype=torch.float64)
tensor([4.9800, 0.2684, 4.7114], dtype=torch.float64)
Latent state
tensor([1.7395, 3.2825, 0.7043], grad_fn=<

tensor([2.0696, 3.7514, 0.6334], grad_fn=<ExpandBackward>)
Observed
tensor([4.9800, 0.2688, 4.7109], dtype=torch.float64)
tensor([4.9800, 0.2688, 4.7109], dtype=torch.float64)
Latent state
tensor([2.0117, 3.6633, 0.6269], grad_fn=<ExpandBackward>)
Observed
tensor([4.9801, 0.2687, 4.7111], dtype=torch.float64)
tensor([4.9801, 0.2687, 4.7111], dtype=torch.float64)
Latent state
tensor([2.0149, 3.6264, 0.6703], grad_fn=<ExpandBackward>)
Observed
tensor([4.9800, 0.2687, 4.7110], dtype=torch.float64)
tensor([4.9800, 0.2687, 4.7110], dtype=torch.float64)
Latent state
tensor([1.9406, 3.5058, 0.6342], grad_fn=<ExpandBackward>)
Observed
tensor([4.9800, 0.2688, 4.7109], dtype=torch.float64)
tensor([4.9800, 0.2688, 4.7109], dtype=torch.float64)
Latent state
tensor([1.8509, 3.3658, 0.5638], grad_fn=<ExpandBackward>)
Observed
tensor([4.9802, 0.2689, 4.7110], dtype=torch.float64)
tensor([4.9802, 0.2689, 4.7110], dtype=torch.float64)
Latent state
tensor([1.8516, 3.3567, 0.6168], grad_fn=<ExpandBackwar

RuntimeError: expected device cpu and dtype Double but got device cpu and dtype Float